# Train a neural network.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Set parameters

In [ ]:
# file with training input and output data: format is x,y
input_file = 'training.csv'

# names of input/output columns
inputs = ['mu', 'angle', 'threshold']
outputs = ['speed']#['low_speed', 'high_speed']

# these set the input/output dimensions of the network
input_size = len(inputs)
output_size = len(outputs)

Read data

In [ ]:
data = pd.read_csv(input_file)
data.columns = inputs + outputs

x = np.array(data[inputs])
y = np.array(data[outputs])

Normalizing preprocessing layer from training data

In [ ]:
normalizer = keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(x)

Base model

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(4, activation='linear',input_dim=input_size,kernel_regularizer=keras.regularizers.l2(0.0000001)))
model.add(keras.layers.Dense(10,activation='relu'))
model.add(keras.layers.Dense(10,activation='relu'))
model.add(keras.layers.Dense(output_size,activation='softplus'))
model.compile(loss='mse',optimizer='adam')

Compile model

In [ ]:
input_shape = x.shape[1:]
full_model_input = keras.Input(shape=input_shape)
normalized_input = normalizer(full_model_input)
full_model_output = model(normalized_input)
full_model = keras.Model(full_model_input, full_model_output)
full_model.compile(loss='mse',optimizer='adam',metrics=['mse'])

Split data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, shuffle= True)

Train model

In [ ]:
model_output = full_model.fit(x_train,y_train,epochs=1000,batch_size=10,verbose=0,validation_data=(x_val,y_val)) # check validation

View output

In [ ]:
plt.plot(model_output.history['loss'])
plt.figure()
plt.plot(model_output.history['mse'])
plt.figure()
plt.plot(model_output.history['val_loss'])
plt.figure()
plt.plot(model_output.history['val_mse'])
plt.figure()

Save model

In [ ]:
full_model.save("2_speed_network.h5")